In [1]:
import requests
import json
import matplotlib
from matplotlib import pyplot as plt 
%matplotlib inline 
import pandas as pd 
import seaborn as sns
import datetime as DT 
import sqlite3
import os.path
from os import path
import numpy as np 
import csv

#maps capital -> country
capitals = {'Sukhumi': 'Abkhazia', 'Kabul': 'Afghanistan', 'Episkopi Cantonment': 'Akrotiri and Dhekelia', 'Tirana': 'Albania', 'Algiers': 'Algeria', 'Pago Pago': 'American Samoa', 'Andorra la Vella': 'Andorra', 'Luanda': 'Angola', 'The Valley': 'Anguilla', "St. John's": 'Antigua and Barbuda', 'Buenos Aires': 'Argentina', 'Yerevan': 'Armenia', 'Oranjestad': 'Aruba', 'Georgetown': 'Guyana', 'Canberra': 'Australia', 'Vienna': 'Austria', 'Baku': 'Azerbaijan', 'Nassau': 'Bahamas', 'Manama': 'Bahrain', 'Dhaka': 'Bangladesh', 'Bridgetown': 'Barbados', 'Minsk': 'Belarus', 'Brussels': 'Belgium', 'Belmopan': 'Belize', 'Porto-Novo': 'Benin', 'Hamilton': 'Bermuda', 'Thimphu': 'Bhutan', 'Sucre': 'Bolivia', 'La Paz': 'Bolivia', 'Sarajevo': 'Bosnia and Herzegovina', 'Gaborone': 'Botswana', 'Brasília': 'Brazil', 'Road Town': 'British Virgin Islands', 'Bandar Seri Begawan': 'Brunei', 'Sofia': 'Bulgaria', 'Ouagadougou': 'Burkina Faso', 'Bujumbura': 'Burundi', 'Phnom Penh': 'Cambodia', 'Yaoundé': 'Cameroon', 'Ottawa': 'Canada', 'Praia': 'Cape Verde', 'George Town': 'Cayman Islands', 'Bangui': 'Central African Republic', "N'Djamena": 'Chad', 'Santiago': 'Chile', 'Beijing': 'China', 'Flying Fish Cove': 'Christmas Island', 'West Island': 'Cocos (Keeling) Islands', 'Bogotá': 'Colombia', 'Moroni': 'Comoros', 'Avarua': 'Cook Islands', 'San José': 'Costa Rica', 'Zagreb': 'Croatia', 'Havana': 'Cuba', 'Willemstad': 'Curaçao', 'Nicosia': 'Northern Cyprus', 'Prague': 'Czech Republic', 'Yamoussoukro': "Côte d'Ivoire", 'Kinshasa': 'Democratic Republic of the Congo', 'Copenhagen': 'Denmark', 'Djibouti': 'Djibouti', 'Roseau': 'Dominica', 'Santo Domingo': 'Dominican Republic', 'Dili': 'East Timor (Timor-Leste)', 'Hanga Roa': 'Easter Island', 'Quito': 'Ecuador', 'Cairo': 'Egypt', 'San Salvador': 'El Salvador', 'Malabo': 'Equatorial Guinea', 'Asmara': 'Eritrea', 'Tallinn': 'Estonia', 'Addis Ababa': 'Ethiopia', 'Stanley': 'Falkland Islands', 'Tórshavn': 'Faroe Islands', 'Palikir': 'Federated States of Micronesia', 'Suva': 'Fiji', 'Helsinki': 'Finland', 'Paris': 'France', 'Cayenne': 'French Guiana', 'Papeete': 'French Polynesia', 'Libreville': 'Gabon', 'Banjul': 'Gambia', 'Tbilisi': 'Georgia', 'Berlin': 'Germany', 'Accra': 'Ghana', 'Gibraltar': 'Gibraltar', 'Athens': 'Greece', 'Nuuk': 'Greenland', "St. George's": 'Grenada', 'Hagåtña': 'Guam', 'Guatemala City': 'Guatemala', 'St. Peter Port': 'Guernsey', 'Conakry': 'Guinea', 'Bissau': 'Guinea-Bissau', 'Port-au-Prince': 'Haiti', 'Tegucigalpa': 'Honduras', 'Budapest': 'Hungary', 'Reykjavík': 'Iceland', 'New Delhi': 'India', 'Jakarta': 'Indonesia', 'Tehran': 'Iran', 'Baghdad': 'Iraq', 'Dublin': 'Ireland', 'Douglas': 'Isle of Man', 'Jerusalem': 'Palestine', 'Rome': 'Italy', 'Kingston': 'Norfolk Island', 'Tokyo': 'Japan', 'St. Helier': 'Jersey', 'Amman': 'Jordan', 'Astana': 'Kazakhstan', 'Nairobi': 'Kenya', 'Tarawa': 'Kiribati', 'Pristina': 'Kosovo', 'Kuwait City': 'Kuwait', 'Bishkek': 'Kyrgyzstan', 'Vientiane': 'Laos', 'Riga': 'Latvia', 'Beirut': 'Lebanon', 'Maseru': 'Lesotho', 'Monrovia': 'Liberia', 'Tripoli': 'Libya', 'Vaduz': 'Liechtenstein', 'Vilnius': 'Lithuania', 'Luxembourg': 'Luxembourg', 'Skopje': 'Macedonia', 'Antananarivo': 'Madagascar', 'Lilongwe': 'Malawi', 'Kuala Lumpur': 'Malaysia', 'Malé': 'Maldives', 'Bamako': 'Mali', 'Valletta': 'Malta', 'Majuro': 'Marshall Islands', 'Nouakchott': 'Mauritania', 'Port Louis': 'Mauritius', 'Mexico City': 'Mexico', 'Chisinau': 'Moldova', 'Monaco': 'Monaco', 'Ulaanbaatar': 'Mongolia', 'Podgorica': 'Montenegro', 'Plymouth': 'Montserrat', 'Rabat': 'Morocco', 'Maputo': 'Mozambique', 'Naypyidaw': 'Myanmar', 'Stepanakert': 'Nagorno-Karabakh Republic', 'Windhoek': 'Namibia', 'Yaren': 'Nauru', 'Kathmandu': 'Nepal', 'Amsterdam': 'Netherlands', 'Nouméa': 'New Caledonia', 'Wellington': 'New Zealand', 'Managua': 'Nicaragua', 'Niamey': 'Niger', 'Abuja': 'Nigeria', 'Alofi': 'Niue', 'Pyongyang': 'North Korea', 'Belfast': 'United Kingdom Northern Ireland', 'Saipan': 'Northern Mariana Islands', 'Oslo': 'Norway', 'Muscat': 'Oman', 'Islamabad': 'Pakistan', 'Ngerulmud': 'Palau', 'Panama City': 'Panama', 'Port Moresby': 'Papua New Guinea', 'Asunción': 'Paraguay', 'Lima': 'Peru', 'Manila': 'Philippines', 'Adamstown': 'Pitcairn Islands', 'Warsaw': 'Poland', 'Lisbon': 'Portugal', 'San Juan': 'Puerto Rico', 'Doha': 'Qatar', 'Taipei': 'Republic of China (Taiwan)', 'Brazzaville': 'Republic of the Congo', 'Bucharest': 'Romania', 'Moscow': 'Russia', 'Kigali': 'Rwanda', 'Gustavia': 'Saint Barthélemy', 'Jamestown': 'Saint Helena', 'Basseterre': 'Saint Kitts and Nevis', 'Castries': 'Saint Lucia', 'Marigot': 'Saint Martin', 'St. Pierre': 'Saint Pierre and Miquelon', 'Kingstown': 'Saint Vincent and the Grenadines', 'Apia': 'Samoa', 'San Marino': 'San Marino', 'Riyadh': 'Saudi Arabia', 'Edinburgh': 'Scotland', 'Dakar': 'Senegal', 'Belgrade': 'Serbia', 'Victoria': 'Seychelles', 'Freetown': 'Sierra Leone', 'Singapore': 'Singapore', 'Philipsburg': 'Sint Maarten', 'Bratislava': 'Slovakia', 'Ljubljana': 'Slovenia', 'Honiara': 'Solomon Islands', 'Mogadishu': 'Somalia', 'Hargeisa': 'Somaliland', 'Pretoria': 'South Africa', 'Grytviken': 'South Georgia and the South Sandwich Islands', 'Seoul': 'South Korea', 'Tskhinvali': 'South Ossetia', 'Juba': 'South Sudan South Sudan', 'Madrid': 'Spain', 'Sri Jayawardenapura Kotte': 'Sri Lanka', 'Khartoum': 'Sudan', 'Paramaribo': 'Suriname', 'Mbabane': 'Swaziland', 'Stockholm': 'Sweden', 'Bern': 'Switzerland', 'Damascus': 'Syria', 'São Tomé': 'São Tomé and Príncipe', 'Dushanbe': 'Tajikistan', 'Dodoma': 'Tanzania', 'Bangkok': 'Thailand', 'Lomé': 'Togo', 'Nukuʻalofa': 'Tonga', 'Tiraspol': 'Transnistria', 'Port of Spain': 'Trinidad and Tobago', 'Edinburgh of the Seven Seas': 'Tristan da Cunha', 'Tunis': 'Tunisia', 'Ankara': 'Turkey', 'Ashgabat': 'Turkmenistan', 'Cockburn Town': 'Turks and Caicos Islands', 'Funafuti': 'Tuvalu', 'Kampala': 'Uganda', 'Kiev': 'Ukraine', 'Abu Dhabi': 'United Arab Emirates', 'London': 'United Kingdom; England', 'Washington, D.C.': 'United States', 'Charlotte Amalie': 'United States Virgin Islands', 'Montevideo': 'Uruguay', 'Tashkent': 'Uzbekistan', 'Port Vila': 'Vanuatu', 'Vatican City': 'Vatican City', 'Caracas': 'Venezuela', 'Hanoi': 'Vietnam', 'Cardiff': 'Wales', 'Mata-Utu': 'Wallis and Futuna', 'El Aaiún': 'Western Sahara', 'Sanaá': 'Yemen', 'Lusaka': 'Zambia', 'Harare': 'Zimbabwe'}

#get list of cities 
cities = list(capitals.keys()) 

#list of 73 most influential countries in the world  
influential_countries = ['Switzerland', 'Canada', 'Japan', 'Germany', 'Australia', 'United Kingdom', 'United States', 'Sweden', 'Netherlands', 'Norway', 'New Zealand', 'France', 'Denmark', 'Finland', 'China', 'Singapore', 'Italy', 'Austria', 'Spain', 'South Korea', 'Luxembourg', 'United Arab Emirates', 'Russia', 'Portugal', 'India', 'Thailand', 'Greece', 'Brazil', 'Israel', 'Qatar', 'Saudi Arabia', 'Malaysia', 'Mexico', 'Poland', 'Turkey', 'Egypt', 'Czech Republic', 'Costa Rica', 'South Africa', 'Morocco', 'Indonesia', 'Argentina', 'Vietnam', 'Croatia', 'Philippines', 'Chile', 'Peru', 'Sri Lanka', 'Dominican Republic', 'Panama', 'Colombia', 'Slovakia', 'Kenya', 'Romania', 'Estonia', 'Slovenia', 'Myanmar', 'Bulgaria', 'Lithuania', 'Bolivia', 'Azerbaijan', 'Latvia', 'Ecuador', 'Jordan', 'Guatemala', 'Kazakhstan', 'Ghana', 'Ukraine', 'Tunisia', 'Belarus', 'Oman', 'Serbia', 'Lebanon']

#classification I made with Continent: Country. Just used a bunch of random techniques to get this. 
classif = {'Algeria': 'Africa', 'Angola': 'Africa', 'Benin': 'Africa', 'Botswana': 'Africa', 'Burkina': 'Africa', 'Burundi': 'Africa', 'Cameroon': 'Africa', 'Cape Verde': 'Africa', 'Central African Republic': 'Africa', 'Chad': 'Africa', 'Comoros': 'Africa', 'Congo': 'Africa', 'Djibouti': 'Africa', 'Egypt': 'Africa', 'Equatorial Guinea': 'Africa', 'Eritrea': 'Africa', 'Ethiopia': 'Africa', 'Gabon': 'Africa', 'Gambia': 'Africa', 'Ghana': 'Africa', 'Guinea': 'Africa', 'Guinea-Bissau': 'Africa', 'Ivory Coast': 'Africa', 'Kenya': 'Africa', 'Lesotho': 'Africa', 'Liberia': 'Africa', 'Libya': 'Africa', 'Madagascar': 'Africa', 'Malawi': 'Africa', 'Mali': 'Africa', 'Mauritania': 'Africa', 'Mauritius': 'Africa', 'Morocco': 'Africa', 'Mozambique': 'Africa', 'Namibia': 'Africa', 'Niger': 'Africa', 'Nigeria': 'Africa', 'Rwanda': 'Africa', 'Sao Tome and Principe': 'Africa', 'Senegal': 'Africa', 'Seychelles': 'Africa', 'Sierra Leone': 'Africa', 'Somalia': 'Africa', 'South Africa': 'Africa', 'South Sudan': 'Africa', 'Sudan': 'Africa', 'Swaziland': 'Africa', 'Tanzania': 'Africa', 'Togo': 'Africa', 'Tunisia': 'Africa', 'Uganda': 'Africa', 'Zambia': 'Africa', 'Zimbabwe': 'Africa', 'Afghanistan': 'Asia', 'Bahrain': 'Asia', 'Bangladesh': 'Asia', 'Bhutan': 'Asia', 'Brunei': 'Asia', 'Myanmar': 'Asia', 'Cambodia': 'Asia', 'China': 'Asia', 'East Timor': 'Asia', 'India': 'Asia', 'Indonesia': 'Asia', 'Iran': 'Asia', 'Iraq': 'Asia', 'Israel': 'Asia', 'Japan': 'Asia', 'Jordan': 'Asia', 'Kazakhstan': 'Asia', 'South Korea': 'Asia', 'Kuwait': 'Asia', 'Kyrgyzstan': 'Asia', 'Laos': 'Asia', 'Lebanon': 'Asia', 'Malaysia': 'Asia', 'Maldives': 'Asia', 'Mongolia': 'Asia', 'Nepal': 'Asia', 'Oman': 'Asia', 'Pakistan': 'Asia', 'Philippines': 'Asia', 'Qatar': 'Asia', 'Russia': 'Asia', 'Saudi Arabia': 'Asia', 'Singapore': 'Asia', 'Sri Lanka': 'Asia', 'Syria': 'Asia', 'Tajikistan': 'Asia', 'Thailand': 'Asia', 'Turkey': 'Asia', 'Turkmenistan': 'Asia', 'United Arab Emirates': 'Asia', 'Uzbekistan': 'Asia', 'Vietnam': 'Asia', 'Yemen': 'Asia', 'Albania': 'Europe', 'Andorra': 'Europe', 'Armenia': 'Europe', 'Austria': 'Europe', 'Azerbaijan': 'Europe', 'Belarus': 'Europe', 'Belgium': 'Europe', 'Bosnia and Herzegovina': 'Europe', 'Bulgaria': 'Europe', 'Croatia': 'Europe', 'Cyprus': 'Europe', 'Czech Republic': 'Europe', 'Denmark': 'Europe', 'Estonia': 'Europe', 'Finland': 'Europe', 'France': 'Europe', 'Georgia': 'Europe', 'Germany': 'Europe', 'Greece': 'Europe', 'Hungary': 'Europe', 'Iceland': 'Europe', 'Ireland': 'Europe', 'Italy': 'Europe', 'Latvia': 'Europe', 'Liechtenstein': 'Europe', 'Lithuania': 'Europe', 'Luxembourg': 'Europe', 'Macedonia': 'Europe', 'Malta': 'Europe', 'Moldova': 'Europe', 'Monaco': 'Europe', 'Montenegro': 'Europe', 'Netherlands': 'Europe', 'Norway': 'Europe', 'Poland': 'Europe', 'Portugal': 'Europe', 'Romania': 'Europe', 'San Marino': 'Europe', 'Serbia': 'Europe', 'Slovakia': 'Europe', 'Slovenia': 'Europe', 'Spain': 'Europe', 'Sweden': 'Europe', 'Switzerland': 'Europe', 'Ukraine': 'Europe', 'United Kingdom': 'Europe', 'Vatican City': 'Europe', 'Antigua and Barbuda': 'North America', 'Bahamas': 'North America', 'Barbados': 'North America', 'Belize': 'North America', 'Canada': 'North America', 'Costa Rica': 'North America', 'Cuba': 'North America', 'Dominica': 'North America', 'Dominican Republic': 'North America', 'El Salvador': 'North America', 'Grenada': 'North America', 'Guatemala': 'North America', 'Haiti': 'North America', 'Honduras': 'North America', 'Jamaica': 'North America', 'Mexico': 'North America', 'Nicaragua': 'North America', 'Panama': 'North America', 'Saint Kitts and Nevis': 'North America', 'Saint Lucia': 'North America', 'Saint Vincent and the Grenadines': 'North America', 'Trinidad and Tobago': 'North America', 'United States': 'North America', 'Australia': 'Oceania', 'Fiji': 'Oceania', 'Kiribati': 'Oceania', 'Marshall Islands': 'Oceania', 'Micronesia': 'Oceania', 'Nauru': 'Oceania', 'New Zealand': 'Oceania', 'Palau': 'Oceania', 'Papua New Guinea': 'Oceania', 'Samoa': 'Oceania', 'Solomon Islands': 'Oceania', 'Tonga': 'Oceania', 'Tuvalu': 'Oceania', 'Vanuatu': 'Oceania', 'Argentina': 'South America', 'Bolivia': 'South America', 'Brazil': 'South America', 'Chile': 'South America', 'Colombia': 'South America', 'Ecuador': 'South America', 'Guyana': 'South America', 'Paraguay': 'South America', 'Peru': 'South America', 'Suriname': 'South America', 'Uruguay': 'South America', 'Venezuela': 'South America'}

#todays date 
today = str(DT.date.today()).split('date')[0]

#top 20 
top20 = ['Switzerland','Canada','Japan','Germany','Australia','United Kingdom','United States','Sweden','Netherlands','Norway','New Zealand','France','Denmark','Finland','China','Singapore','Italy','Austria','Spain','South Korea']


#top20 list for covid-19 cases class 
top20forfinalcountrycases = ['Switzerland','Canada','Japan','Germany','Australia','UK','USA','Sweden','Netherlands','Norway','New Zealand','France','Denmark','Finland','China','Singapore','Italy','Austria','Spain','S. Korea']

#create 'data' database 
if path.exists('data.db') is False: 
    conn = sqlite3.connect('data.db')
    conn.close() 


# Weather Data 

In [2]:
class WeatherData: 
    
    def __init__(self): 
        """
        Initializes each Weather Data object. Creates city_urls attribute which contains all request urls for all capital cities around the world. 
        """
        city_urls = []
        for city in cities: 
            city_urls.append(self.create_request_url_weather(city))
            
        #save as attribute 
        self.city_urls = city_urls
     
        
    #API Key #1: 3283190565ac48b9bfff13e1b9b319c1
    #API Key #2: eedbb9c044e94669ba9bc59b4e1e7a6e
    def create_request_url_weather(self, city):  
        """
        Takes in a city, returns request URL. 
        """
        url = 'https://api.weatherbit.io/v2.0/current?city='+ city + '&key=' + 'eedbb9c044e94669ba9bc59b4e1e7a6e'
        return url 


    def fit_weather_list(self, country_weather): 
        """
        Take in a dictionary 'country_weather'. Returns a dictionary with fitted list. 
        """
        #make the following edits: #'United Kingdom; England' -> United Kingdom, Palestine -> Israel, United States Virgin Islands -> United States 
        deletelist = ['United Kingdom; England', 'Palestine', 'United States Virgin Islands']
        countryfixlist = ['United Kingdom', 'Israel', 'United States'] 
        datafixlist = []
        returndict = {}

        #grab corresponding values for the keys that you are about to delete, create datafixlist 
        for country in deletelist:
            try: 
                datafixlist.append(country_weather[country]) 
            except: 
                continue 
        #delete all wrongly named countries 
        for country in deletelist: 
            try: 
                del country_weather[country]
            except: 
                continue 

        #reappend the correct key value pairs with correct country nomenclature 
        for i in range(len(datafixlist)):
            try: 
                country_weather[countryfixlist[i]] = datafixlist[i]
            except: 
                continue 
        #remove all countries not considered influential 
        for country in influential_countries: 
            try: 
                returndict[country] = country_weather[country]
            except: 
                continue 
        return returndict 

    def get_weather_data(self): 
        """
        Gets weather data using the list of urls, returns a dictionary called country_weather. 
        """
        country_weather = {}
        for url in self.city_urls:
            try: 
                #requests returns a response object, convert response content to dictionary 
                data = requests.get(url).json()
                #grab the city and grab the corresponding country from capitals[dict]
                city = url.split('=')[1].split('&')[0].strip()
                country = capitals[city]
                weather = float(data['data'][0]['temp']) 
                country_weather[country] = weather
            except: 
                continue
            
        country_weather = self.fit_weather_list(country_weather)
        return country_weather
    
    
    def create_insert_weather_tuple(self, country_weather):
        """
        Takes in a dictionary, returns a tuple of values with only 20 top most influential countries to insert into database. 
        """
        top20tuplefinal = []
        insertweathertuple = []
        
        
        #create a list from the top 20 tuple     
        top20tuple = list(country_weather.items()) 

        #create a final list of tuples that ONLY CONTAIN TOP 20 countries chosen, convert -> dataframe 
        for i in range(len(top20tuple)):
            if top20tuple[i][0] in top20: 
                top20tuplefinal.append(top20tuple[i])

        for tup in top20tuplefinal: 
            insertweathertuple.append((today, tup[0], tup[1])) 
        return insertweathertuple
    
    #run everytime you want to create/append 
    def create_and_append_weather(self, insertweathertuple): 
        """
        Grab data and append data to country_weather table in data database. No return value.  
        """
        #connect/create database
        conn = sqlite3.connect('data.db')
        conn.execute('''CREATE TABLE if not exists country_weather (date text, country text, Fahrenheit FLOAT)''')
        conn.executemany('INSERT INTO country_weather VALUES (?,?,?)', insertweathertuple)
        #print output 
        for row in conn.execute('SELECT * FROM country_weather'):
            print(row)
        conn.commit()
        conn.close() 




def convert_to_fahrenheit(dictionary):
    """
    Convert a dictionary from celsius to fahrenheit by modifying the values. Returns dictionary with all values in fahrenheit. 
    """
    for key in dictionary.keys(): 
        dictionary[key] = dictionary[key]*9/5 + 32 
    return dictionary


#UN-COMMENT BELOW TO ADD DATA. RUN THIS ONCE A DAY! FOR 5 days!
weatherobj = WeatherData() 
country_weather = weatherobj.get_weather_data()
country_weather = convert_to_fahrenheit(country_weather)
final_country_weather = country_weather
weatherobj.create_and_append_weather(weatherobj.create_insert_weather_tuple(country_weather))

('2020-03-29', 'Switzerland', 37.04)
('2020-03-29', 'Canada', 68.0)
('2020-03-29', 'Japan', 35.96)
('2020-03-29', 'Germany', 32.36)
('2020-03-29', 'Australia', 60.08)
('2020-03-29', 'United Kingdom', 75.02)
('2020-03-29', 'United States', 78.98)
('2020-03-29', 'Sweden', 33.08)
('2020-03-29', 'Netherlands', 35.96)
('2020-03-29', 'Norway', 37.04)
('2020-03-29', 'New Zealand', 77.72)
('2020-03-29', 'France', 42.08)
('2020-03-29', 'Denmark', 35.96)
('2020-03-29', 'Finland', 28.94)
('2020-03-29', 'China', 35.6)
('2020-03-29', 'Singapore', 80.6)
('2020-03-29', 'Italy', 53.96)
('2020-03-29', 'Austria', 84.38000000000001)
('2020-03-29', 'Spain', 68.0)
('2020-03-29', 'South Korea', 35.6)
('2020-03-30', 'Switzerland', 44.96)
('2020-03-30', 'Canada', 64.04)
('2020-03-30', 'Japan', 44.96)
('2020-03-30', 'Germany', 37.04)
('2020-03-30', 'Australia', 62.059999999999995)
('2020-03-30', 'United Kingdom', 57.2)
('2020-03-30', 'United States', 82.4)
('2020-03-30', 'Sweden', 30.92)
('2020-03-30', 'Nether

# COVID-19 Data 

In [3]:
class COVIDCases:  
    #refit a dictionary to the 73 most influential countries
    def fit_list(self, dictionary): 
        """
        Takes in a dictionary, returns a dictionary with corrected key names. 
        """
        returndict = {}
        #make the following edits: Czechia -> Czech Republic, S.Korea -> South Korea, UAE  -> United Arab Emirates, UK -> United Kingdom, USA -> United States 
        deletelist = ['Czechia', 'S. Korea', 'UAE', 'UK', 'USA']
        countryfixlist = ['Czech Republic', 'South Korea', 'United Arab Emirates', 'United Kingdom', 'United States']
        datafixlist = []

        #grab corresponding values for the keys that you are about to delete, create datafixlist
        for country in deletelist: 
            datafixlist.append(dictionary[country]) 
        #delete all wrongly named countries 
        for country in deletelist: 
            del dictionary[country]
        #reappend the correct key value pairs with correct country nomenclature 
        for i in range(len(datafixlist)): 
            dictionary[countryfixlist[i]] = datafixlist[i]
        #remove all countries not considered influential 
        for country in influential_countries:
            returndict[country] = dictionary[country]
        return returndict

    
    #get the country: case dictionary for all 73 influential countries 
    def get_country_cases_dict(self): 
        """
        Returns a dictionary for 73 most influential countries and corresponding number of cases. 
        """
        url = "https://coronavirus-monitor.p.rapidapi.com/coronavirus/cases_by_country.php"
        headers = {
            'x-rapidapi-host': "coronavirus-monitor.p.rapidapi.com",
            'x-rapidapi-key': "c902dd2b74mshf8b7588307400dap1d10c3jsnd41616838035"
            }

        country_cases = requests.request("GET", url, headers=headers).json() 

        final_country_cases = {}

        for i in range(len(country_cases['countries_stat'])): 
            #make a final dictionary with country_name -> number of cases 
            final_country_cases[country_cases['countries_stat'][i]['country_name']] = float(country_cases['countries_stat'][i]['cases'].replace(',','').strip()) 
        final_country_cases = self.fit_list(final_country_cases)
        return final_country_cases


    def get_covid_info_by_country_today(self, country): 
        """
        Takes in country, returns COVID data for that country.  
        """
        table = {}
        final_dict = {}
        listofdicts = []

        url = "https://coronavirus-monitor.p.rapidapi.com/coronavirus/history_by_particular_country_by_date.php"

        headers = {
        'x-rapidapi-host': "coronavirus-monitor.p.rapidapi.com",
        'x-rapidapi-key': "c902dd2b74mshf8b7588307400dap1d10c3jsnd41616838035"
        }

        querystring = {"country": country, "date": today}

        response = requests.request("GET", url, headers=headers, params=querystring).json() 
        
        #need to loop through and average by date. need to make date look prettier.
        table['Date'] = today
        try: 
            table['Total Cases'] = float(response['stat_by_country'][0]['total_cases'].replace(',',''))
        except: 
            pass
        try: 
            table['Total Deaths'] = float(response['stat_by_country'][0]['total_deaths'].replace(',',''))
        except: 
            pass 
        try: 
            table['Total Recovered'] = float(response['stat_by_country'][0]['total_recovered'].replace(',',''))
        except: 
            pass 
        try: 
            table['New Cases'] = float(response['stat_by_country'][0]['new_cases'].replace(',',''))
        except: 
            pass
        try: 
            table['New Deaths'] = float(response['stat_by_country'][0]['new_deaths'].replace(',',''))
        except:
            pass
        try: 
            table['Active Cases'] = float(response['stat_by_country'][0]['active_cases'].replace(',',''))
        except: 
            pass 
        try: 
            table['Country'] = country
        except: 
            pass 

        return table 
    
    def create_and_insert_cases_tuple(self): 
        """
        Creates/returns the tuple for 20 most influential countries to insert into database. 
        """
        dataframe = []
        #convert to a dataframe
        for country in top20forfinalcountrycases: 
            dataframe.append(self.get_covid_info_by_country_today(country))
        #create dataframe 
        casesdf = pd.DataFrame(dataframe)  
        #replace the names to be consistent with the full nomenclature 
        try: 
            casesdf["Country"]= casesdf["Country"].replace('USA', "United States") 
        except: 
            pass
        try: 
            casesdf["Country"]= casesdf["Country"].replace('S. Korea', "South Korea") 
        except: 
            pass
        try: 
            casesdf["Country"]= casesdf["Country"].replace('UK', "United Kingdom") 
        except:
            pass 
        #create insertcasestuple 
        insertcasestuple = list(map(tuple, casesdf.to_numpy()))
        return insertcasestuple 


    #run everytime you want to create/append 
    def create_and_append_cases(self, insertcasestuple): 
        """
        Take in tuple, append data to cases table in data database. 
        """
        #connect/create database
        conn = sqlite3.connect('data.db')
        conn.execute('''CREATE TABLE if not exists cases (date text, total_cases float, total_deaths float, total_recovered float, new_cases float, new_deaths float, active_cases float, country text)''')
        conn.executemany('INSERT INTO cases VALUES (?,?,?,?,?,?,?,?)', insertcasestuple)
        #print output  
        for row in conn.execute('SELECT * FROM cases'):
            print(row)

        conn.commit() 
        conn.close() 


# UN-COMMENT BELOW TO ADD CASES RUN THIS ONCE A DAY! FOR 5 days! Takes 5-10 minutes to run due to latency from API. 
casesobj = COVIDCases() 
casesobj.create_and_append_cases(casesobj.create_and_insert_cases_tuple())
# UN-COMMENT BELOW IF YOU HAVE A LOT OF TIME TO WAIT. 
# final_country_cases = casesobj.get_country_cases_dict()

('2020-03-29', 14076.0, 14076.0, 1530.0, 1148.0, 33.0, 12282.0, 'Switzerland')
('2020-03-29', 5655.0, 5655.0, 396.0, 898.0, 5.0, 5199.0, 'Canada')
('2020-03-29', 1693.0, 1693.0, 404.0, 194.0, 3.0, 1237.0, 'Japan')
('2020-03-29', 57695.0, 57695.0, 8481.0, 6824.0, 82.0, 48781.0, 'Germany')
('2020-03-29', 3635.0, 3635.0, 170.0, 257.0, 1.0, 3451.0, 'Australia')
('2020-03-29', 17089.0, 17089.0, 135.0, 2546.0, 260.0, 15935.0, 'United Kingdom')
('2020-03-29', 123313.0, 123313.0, 3231.0, 19187.0, 515.0, 117871.0, 'United States')
('2020-03-29', 3447.0, 3447.0, 16.0, 378.0, None, None, None)
('2020-03-29', 9762.0, 9762.0, 3.0, 1159.0, 93.0, 9120.0, 'Netherlands')
('2020-03-29', 4015.0, 4015.0, 7.0, 244.0, 4.0, 3985.0, 'Norway')
('2020-03-29', 451.0, 451.0, 50.0, 83.0, None, None, None)
('2020-03-29', 37575.0, 37575.0, 5700.0, 4611.0, 319.0, 29561.0, 'France')
('2020-03-29', 2201.0, 2201.0, 1.0, 155.0, 13.0, 2135.0, 'Denmark')
('2020-03-29', 1167.0, 1167.0, 10.0, 126.0, 2.0, 1148.0, 'Finland')
(

# Stock Data 

In [4]:
class Stocks: 
    #API KEY: M2OPINUK58M3497T
    #https://robinhood.com/collections/100-most-popular

    def __init__(self, tickerlist): 
        """
        Initializes Stock object. Creates past_20_days attribute, a list of the past 20 dates. 
        """
        self.tickerlist = tickerlist 
        self.past_20_days = self.get_past_20_days()
    def get_past_20_days(self): 
        """
        Return a list of the past 20 dates 
        """
        past_20_days = []
        today = DT.date.today()

        for i in range(1,21): 
            day_ago = today - DT.timedelta(days=i)
            past_20_days.append(str(day_ago).split('date')[0]) 

        return past_20_days 


    def get_last_20_days_stock_data(self):  
        """
        Uses the stock object attribute to create a request URL for the past 20 dates, return a list of tuples with that stock
        for the past 20 days. 
        """
        listoftuples = []
        for ticker in self.tickerlist: 
            url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+ ticker + '&outputsize=compact&apikey=M2OPINUK58M3497T'
            response = requests.get(url).json() 

            for date in self.past_20_days: 
                try: 
                    listoftuples.append((date, ticker, float(response['Time Series (Daily)'][date]['4. close']))) 
                except: 
                    continue 

        return listoftuples 
    
    #run everytime you want to create/append 
    def create_and_append_stocks(self, inserttuple): 
        """
        Take in a tuple, append the data inside the tuple into a stocks table. 
        """
        #connect/create database
        conn = sqlite3.connect('data.db')
        conn.execute('''CREATE TABLE if not exists stocks (date text, ticker text, price float)''')
        conn.executemany('INSERT INTO stocks VALUES (?,?,?)', inserttuple)
        #print output  
        for row in conn.execute('SELECT * FROM stocks'):
            print(row)

        conn.commit() 
        conn.close() 

#UN-COMMENT below if you would like to add different stock's data for the last 20 days. 

stockinput = input('Enter a stock ticker: ')
stock = Stocks([stockinput])
stock.create_and_append_stocks(stock.get_last_20_days_stock_data())  





KeyboardInterrupt: Interrupted by user

# Music Data 


In [5]:
top100 = ['Norway', 'Switzerland','Australia','Ireland','Germany','Iceland','Sweden','Hong Kong','Singapore','Netherlands','Denmark','Canada','United States','United Kingdom','Finland','New Zealand','Belgium', 'Liechtenstein', 'Japan', 'Austria', 'Luxembourg', 'Haiti', 'Israel', 'France', 'Slovenia', 'Spain','Italy','Malta','Estonia', 'Greece', 'Cyprus', 'Poland', 'United Arab Emirates','Lithuania', 'Andorra', 'Qatar', 'Slovakia','Nepal', 'Saudi Arabia', 'Portugal', 'Latvia', 'Bahrain', 'Chile', 'Hungary', 'Croatia', 'Argentina', 'Oman', 'Chad', 'Montenegro', 'Bulgaria', 'Romania', 'Belarus', 'Bahamas', 'Uruguay', 'Kuwait', 'Malaysia', 'Barbados', 'Kazakhstan', 'Palau', 'Angola', 'Seychelles', 'Costa Rica', 'Turkey', 'Mauritius', 'Panama', 'Serbia', 'Albania', 'Trinidad and Tobago', 'Antigua and Barbuda', 'Georgia', 'Saint Kitts and Nevis', 'Cuba', 'Mexico', 'Grenada', 'Sri Lanka', 'Bosnia and Herzegovina', 'Venezuela', 'Brazil', 'Lebanon', 'Azerbaijan', 'Thailand', 'Armenia', 'Algeria', 'Ecuador', 'China', 'Ukraine', 'Peru', 'Saint Lucia', 'Colombia', 'Mongolia', 'Fiji', 'Dominican Republic', 'Tunisia', 'Jordan', 'Jamaica', 'Tonga', 'Saint Vincent and the Grenadines', 'Suriname', 'Botswana', 'Maldives']
twentyone = []
twentytwo = []
twentythree = []
twentyfour = []
twentyfive = []

twentyone = top100[0:20]

twentytwo= top100[20:40]

twentythree= top100[40:60]

twentyfour= top100[60:80]

twentyfive= top100[80:100]



#take in user input 1-5 to grab different sets of data with manually changing the code each run. 
userinput = input('Enter a 1,2,3,4,5 (to choose what list of 20 countries) : ')


APIKEY = '010c0f8122235fc473fbb31980871052'
def request_url_geo_top_artists(country):
    """
    Pass in the country. Returns request url for geo top artists API endpoint. 
    """
    url = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&country=' + country + '&api_key=' + APIKEY + '&format=json'
    return url 


def request_url_geo_top_tracks(country): 
    """
    Pass in the country. Returns request url for geo top tracks API endpoint. 
    """
    url = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettoptracks&country=' + country + '&api_key=' + APIKEY + '&format=json'
    return url 



def get_top_artists(request_url): 
    """
    Pass in a request url. Returns a list of tuples to prepare data for database and visualization. 
    """    
    data = requests.get(request_url).json()
    
    country = request_url.split('&')[1].split('=')[1]
    
    
    artist = data['topartists']['artist'][0]['name']
    listeners = float(data['topartists']['artist'][0]['listeners']) 
    artistdata = requests.get('http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist=' + artist + '&api_key=010c0f8122235fc473fbb31980871052&format=json').json() 
    genre = artistdata['artist']['tags']['tag'][0]['name']
    
    tuplelist = [(today, country, artist, listeners, genre)]
    return tuplelist 

def get_top_tracks(request_url): 
    """
    Pass in a request url. Returns a list of tuples to prepare data for database and visualization. 
    """
    data = requests.get(request_url).json()
    country = request_url.split('&')[1].split('=')[1]
    trackname = data['tracks']['track'][0]['name'] 
    listeners = float(data['tracks']['track'][0]['listeners']) 
    artist = data['tracks']['track'][0]['artist']['name']
    genredata = requests.get('http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key=010c0f8122235fc473fbb31980871052&artist=' + artist + '&track=' + trackname + '&format=json').json()
    genre = genredata['track']['toptags']['tag'][0]['name']
    duration = float(genredata['track']['duration'])/1000
    tuplelist = [(today, country, trackname, duration, listeners, artist, genre)]    
    return tuplelist    

    


#list of urls 
urllist = [] 
urllist2 = []


#first list of countries 1-20 
if int(userinput) == 1:
    
     
    artist_data = []
    track_data = []

    
    #make a list of all the request urls
    for country in twentyone: 
        urllist.append(request_url_geo_top_artists(country)) 
        
    for url in urllist: 
        artist_data += get_top_artists(url)
    
    
    #make a list of all the request urls
    for country in twentyone: 
        urllist2.append(request_url_geo_top_tracks(country))
    
    
    #grab the list of one tuple. add each entry to a final list 
    for url in urllist2: 
        track_data += get_top_tracks(url) 
    
    
    print('Artist Data: ' + str(artist_data))
    print('Track Data: ' + str(track_data))  
    
#second list of countries 21-40 
elif int(userinput) == 2:
    
    artist_data = []
    track_data = []

    #make a list of all the request urls
    for country in twentytwo: 
        urllist.append(request_url_geo_top_artists(country))
    for url in urllist: 
        artist_data += get_top_artists(url)
        
    #make a list of all the request urls
    for country in twentytwo: 
        urllist2.append(request_url_geo_top_tracks(country))
    
    
    #grab the list of one tuple. add each entry to a final list 
    for url in urllist2: 
        track_data += get_top_tracks(url) 
        
    print('Artist Data: ' + str(artist_data))
    print('Track Data: ' + str(track_data))  
    
#third list of countries 41-60
elif int(userinput) == 3: 
    
    artist_data = []
    track_data = []

    #make a list of all the request urls
    for country in twentythree: 
        urllist.append(request_url_geo_top_artists(country))
        
    for url in urllist: 
        artist_data += get_top_artists(url)
    
    #make a list of all the request urls
    for country in twentythree: 
        urllist2.append(request_url_geo_top_tracks(country))
    
    
    #grab the list of one tuple. add each entry to a final list 
    for url in urllist2: 
        track_data += get_top_tracks(url) 
        
        
    
    
    print('Artist Data: ' + str(artist_data))
    print('Track Data: ' + str(track_data))  
    
    
#4th list of countries 61-80
elif int(userinput) == 4: 
    
    artist_data = []
    track_data = []

    #make a list of all the request urls
    for country in twentyfour:
        urllist.append(request_url_geo_top_artists(country))
    for url in urllist: 
        artist_data += get_top_artists(url)
        
        
    #make a list of all the request urls
    for country in twentyfour: 
        urllist2.append(request_url_geo_top_tracks(country))
    
    
    #grab the list of one tuple. add each entry to a final list 
    for url in urllist2: 
        track_data += get_top_tracks(url) 
        
        
    print('Artist Data: ' + str(artist_data))
    print('Track Data: ' + str(track_data))  
    

#5th list of countries 81-100 
elif int(userinput) == 5: 
    
    artist_data = []
    track_data = []

    #make a list of all the request urls
    for country in twentyfive:
        urllist.append(request_url_geo_top_artists(country))
    
    #grab the list of one tuple. add each entry to a final list 
    for url in urllist: 
        artist_data += get_top_artists(url)
        
            
    #make a list of all the request urls
    for country in twentyfive: 
        urllist2.append(request_url_geo_top_tracks(country))
    
    
    #grab the list of one tuple. add each entry to a final list 
    for url in urllist2: 
        track_data += get_top_tracks(url) 
        
    print('Artist Data: ' + str(artist_data))
    print('Track Data: ' + str(track_data))  

    
#run everytime you want to create/append 
def create_and_append_song_data(): 
    """
    Creates tracks & artists table if they don't exist. Append data to tracks and artists table. Returns nothing. 
    """
    #connect/create database
    conn = sqlite3.connect('data.db')
    
    #creating and inserting into top artists table 

    conn.execute('''CREATE TABLE IF NOT EXISTS artists(date text, country text, artist text, listeners float, genre text)''')
    conn.executemany('INSERT INTO artists VALUES (?,?,?,?,?)', artist_data)
    
    #creating and inserting into top tracks table 
    conn.execute('''CREATE TABLE IF NOT EXISTS tracks(date text, country text, track text, duration float, listeners float, artist text, genre text)''')
    conn.executemany('INSERT INTO tracks VALUES (?,?,?,?,?,?,?)', track_data)
    
    #both the artists and tracks data share a joint key of the date and the country. 
    
    #print output  
    for row in conn.execute('SELECT * FROM tracks'):
        print(row)
    for row in conn.execute('SELECT * FROM artists'):
        print(row)
    

    conn.commit() 
    conn.close() 


#UN-COMMENT BELOW IF YOU WOULD LIKE TO APPEND SONG DATA 
# create_and_append_song_data() 

Enter a 1,2,3,4,5 (to choose what list of 20 countries) : 1
Artist Data: [('2020-04-18', 'Norway', 'Coldplay', 5521938.0, 'rock'), ('2020-04-18', 'Switzerland', 'Coldplay', 5521938.0, 'rock'), ('2020-04-18', 'Australia', 'Radiohead', 4856426.0, 'alternative'), ('2020-04-18', 'Ireland', 'David Bowie', 3477542.0, 'rock'), ('2020-04-18', 'Germany', 'Queen', 4177952.0, 'classic rock'), ('2020-04-18', 'Iceland', 'David Bowie', 3477542.0, 'rock'), ('2020-04-18', 'Sweden', 'David Bowie', 3477542.0, 'rock'), ('2020-04-18', 'Hong Kong', 'Coldplay', 5521938.0, 'rock'), ('2020-04-18', 'Singapore', 'Coldplay', 5521938.0, 'rock'), ('2020-04-18', 'Netherlands', 'Coldplay', 5521938.0, 'rock'), ('2020-04-18', 'Denmark', 'Queen', 4177952.0, 'classic rock'), ('2020-04-18', 'Canada', 'Radiohead', 4856426.0, 'alternative'), ('2020-04-18', 'United States', 'Radiohead', 4856426.0, 'alternative'), ('2020-04-18', 'United Kingdom', 'David Bowie', 3477542.0, 'rock'), ('2020-04-18', 'Finland', 'Queen', 4177952.0

# Querying, Calculating, & Visualizations 

In [11]:
class Calculate: 


    def avg_temp_by_country(self): 
        """
        Calculates the average temperature by country. Returns a tuple that contains the query result. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT AVG(Fahrenheit) as Avg_Temp_F, country FROM country_weather WHERE country <> 'None' GROUP BY country'''):
            result.append(row)
        conn.commit() 
        conn.close()
        return result 
    
    def historic_cases(self): 
        """
        Calculates the cases by date and country. Returns a tuple that contains the query result. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT date, country, MAX(total_cases) FROM cases WHERE country <> 'None' GROUP BY date, country'''):
            result.append(row)
        conn.commit() 
        conn.close()
        return result 
    
    
    def avg_new_cases_per_day_by_country(self):
        """
        Calculates the average new cases by country. Returns a tuple that contains the query result. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT country, AVG(new_cases) FROM cases WHERE country <> 'None' GROUP BY country'''):
            result.append(row)
        conn.commit() 
        conn.close()
        return result 
    

    def join_calc(self): 
        """
        Calculates using a JOIN the average weather, total number of cases. Returns a tuple that contains the query result. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT AVG(Fahrenheit) as Avg_Temp, MAX(total_cases) as total_cases, w.country FROM country_weather w INNER JOIN cases c on w.country == c.country WHERE w.country <> 'None' GROUP by w.country'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result 
    def avg_stock_price(self): 
        """
        Calculates average stock price per stock. Returns a tuple that contains the query result. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT ticker, AVG(price) FROM stocks GROUP BY ticker'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result 
    
    
    def stocks(self): 
        """
        Select all data from the stocks table. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT * FROM stocks'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result
    
    def check_length_cases(self): 
        """
        Checks the number of entries of the cases table. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT COUNT(date) FROM cases'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result 
    def check_length_stocks(self): 
        """
        Checks the number of entries of the stocks table. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT COUNT(date) FROM stocks'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result

    def check_length_weather(self): 
        """
        Checks the number of entries of the country_weather table. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT COUNT(date) FROM country_weather'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result
    def check_length_tracks(self): 
        """
        Checks the number of entries of the tracks table. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT COUNT(date) FROM tracks'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result
    
    def check_length_artists(self): 
        """
        Checks the number of entries of the artists table. 
        """
        result = []
        conn = sqlite3.connect('data.db')
        for row in conn.execute('''SELECT COUNT(date) FROM artists'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result
    

    def artist_calculate(self): 
        """
        Returns the artist, and number of times that artist shows up 
        """
        result = []
        conn = sqlite3.connect('data.db')

        #count of the number of artists 
        for row in conn.execute('''SELECT artist, count(artist) FROM artists GROUP BY artist ORDER BY count(artist) desc'''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result


    def track_calculate(self): 
        """
        Returns country, cases, # listeners, artist, and genre 
        """
        result = []
        conn = sqlite3.connect('data.db')

        #count of the number of artists 
        for row in conn.execute('''SELECT c.country, max(total_cases), max(listeners), artist, genre FROM artists a inner join cases c on a.country = c.country WHERE c.country <> 'None' GROUP by c.country '''):
            result.append(row) 
        conn.commit() 
        conn.close()
        return result


        


#create Calculate object named calc  
calc = Calculate() 

calc.drop() 

#calculate the artist and the number of times the artist appears 
artistcount = calc.artist_calculate() 


#create artist dataframe  
artistcountdf = pd.DataFrame.from_records(artistcount, columns = ['Artist', 'Artist Count'])

#grab the cases, # listeners, artist and genre
trackcount = calc.track_calculate() 

#create track dataframe  
trackcountdf = pd.DataFrame.from_records(trackcount, columns = ['Country', '# of COVID-19 Cases', '# of Listeners',  'Artist', 'Genre'])

#create scatter plot of number of listeners, number of covid-19 cases, and genre 
sns.scatterplot(x="# of Listeners", y="# of COVID-19 Cases", hue = 'Genre', data=trackcountdf)
plt.tight_layout()
plt.savefig('numcases_vs_listeners_by_genre.pdf')
plt.clf() 


#create artist count visualization
a4_dims = (35, 30)
fig, ax = plt.subplots(figsize=a4_dims)
sns.set(style="whitegrid")
sns.barplot(ax = ax, x=artistcountdf['Artist'], y=artistcountdf['Artist Count'])
plt.savefig('topartistcount.pdf')
plt.clf() 



#create dataframes for above queries 
joindf = pd.DataFrame.from_records(calc.join_calc(), columns =['Average Temperature of Different Countries (F)', 'Current # of Cases COVID-19', 'Country']) 
hcasesdf = pd.DataFrame.from_records(calc.historic_cases(), columns = ['Date', 'Country', 'Total # of COVID-19 Cases'])
newcasesdf = pd.DataFrame.from_records(calc.avg_new_cases_per_day_by_country(), columns = ['Country', 'Average # of New COVID-19 Cases'])
stocksdf = pd.DataFrame.from_records(calc.avg_stock_price(), columns = ['Ticker', 'Average Price'])


sns.distplot(stocksdf['Average Price'])
plt.savefig('average_stock_price.pdf')
plt.clf() 

sns.set(style="darkgrid")

a4_dims = (30, 30)
fig, ax = plt.subplots(figsize=a4_dims)

# Plot the responses for different events and regions
sns.barplot(ax = ax, x=hcasesdf["Country"], y=hcasesdf["Total # of COVID-19 Cases"])
plt.savefig('histogram_cases.pdf')
plt.clf()

# Plot the responses for different events and regions
sns.barplot(x=newcasesdf["Country"], y=newcasesdf["Average # of New COVID-19 Cases"])
plt.savefig('histogram_new_cases.pdf')
plt.clf()

#graph time series for stocks 
a4_dims = (25, 25)
fig, ax = plt.subplots(figsize=a4_dims)
stocktimeseriesdf = pd.DataFrame.from_records(calc.stocks(), columns =['Date', 'Ticker', 'Price']) 
sns.lineplot(ax = ax, x=stocktimeseriesdf["Date"], y=stocktimeseriesdf['Price'], hue = stocktimeseriesdf['Ticker'])
plt.savefig('stock_time_series.pdf')
plt.clf() 


# Avg Temp vs. Current # Deaths 
sns.regplot(x=joindf["Average Temperature of Different Countries (F)"], y=joindf["Current # of Cases COVID-19"], color = 'green') 
plt.savefig('avg_temp_num_cases.pdf')
plt.clf()



        


#write out avg_temp_by_country calculation to text/csv file                                
with open('avg_temp_by_country.csv','w') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['Temperature (F)','Country'])
    for row in calc.avg_temp_by_country():
        csv_out.writerow(row)

#write out temperature vs deaths COVID-19 calculation to text/csv file                                
with open('temp_vs_cases.csv','w') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['Average Temperature (F)','Current # of Cases COVID-19', 'Country'])
    for row in calc.join_calc(): 
        csv_out.writerow(row)

#write out average number of new cases 
with open('avg_new_cases.csv','w') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['Country','Average # of New COVID-19 Cases'])
    for row in calc.avg_new_cases_per_day_by_country(): 
        csv_out.writerow(row)
    
#write out average stock price per ticker chosen 
with open('avg_stock_price.csv','w') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['Ticker','Average Stock Price'])
    for row in calc.avg_stock_price(): 
        csv_out.writerow(row)



#write out covid-19 vs. listeners per genre  
with open('numcases_vs_listeners_by_genre.csv','w') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['Country', '# of COVID-19 Cases', '# of Listeners',  'Artist', 'Genre'])
    for row in trackcount: 
        csv_out.writerow(row)


#write out artist and corresponding count 
with open('artistcount.csv','w') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['Artist','Artist Count'])
    for row in artistcount: 
        csv_out.writerow(row)





#check the length of all the tables to check for 100 items. they all have at least 100 items. 
print('Length of Cases Table' + str(calc.check_length_cases()))  
print('Length of Stocks Table' + str(calc.check_length_stocks())) 
print('Length of Weather Table' + str(calc.check_length_weather())) 
print('Length of Tracks Table' + str(calc.check_length_tracks())) 
print('Length of Artists Table' + str(calc.check_length_artists())) 



# #BELOW contains the pair plot and the correlation graph with continents. 
#if you have a lot of time to wait around, 
#you can un-comment the COVID-19 cases line way above. this creates two insightful charts attached in the report. 


# data = []
# clist = []

# #make the columns the country so its visible 
# clist = sorted(list(final_country_weather.keys()))  
    
# #make list of lists to ready data for pandas
# for country in final_country_weather.keys(): 
#     #weather, cases, classification
#     data.append([final_country_weather[country],final_country_cases[country], classif[country]]) 
    

# #load into pandas 
# continentdf = pd.DataFrame(data, columns = ['Weather (F)', 'Current # of COVID-19 Cases', 'Continent']) 

# sns.set_style('white') 
# sns.set_style('ticks')

# #graph the plots 
# lm = sns.lmplot(x= 'Weather (F)', y = 'Current # of COVID-19 Cases', hue = 'Continent', data = continentdf, height=5, aspect=1.6)
# axes = lm.axes

# #set the limit to make the graph look nicer 
# axes[0,0].set_ylim(0,50000)
# plt.savefig('Weather_vs_Cases_regression.pdf')
# plt.clf() 


# #graph pair plots of cases vs weather 
# pair = sns.pairplot(continentdf, hue = 'Continent', height = 4, aspect = 1.5) 
# axespair = pair.axes
# axespair[1,1].set_xlim(0,10000)
# plt.savefig('Weather_vs_Cases_pair_plot.pdf')
# plt.clf()

/home/olili/.local/lib/python3.6/site-packages/matplotlib/backends/_backend_pdf_ps.py:62: RuntimeWarning: Glyph 23431 missing from current font.
  font.set_text(s, 0.0, flags=ft2font.LOAD_NO_HINTING)
/home/olili/.local/lib/python3.6/site-packages/matplotlib/backends/_backend_pdf_ps.py:62: RuntimeWarning: Glyph 22810 missing from current font.
  font.set_text(s, 0.0, flags=ft2font.LOAD_NO_HINTING)
/home/olili/.local/lib/python3.6/site-packages/matplotlib/backends/_backend_pdf_ps.py:62: RuntimeWarning: Glyph 30000 missing from current font.
  font.set_text(s, 0.0, flags=ft2font.LOAD_NO_HINTING)
/home/olili/.local/lib/python3.6/site-packages/matplotlib/backends/_backend_pdf_ps.py:62: RuntimeWarning: Glyph 12498 missing from current font.
  font.set_text(s, 0.0, flags=ft2font.LOAD_NO_HINTING)
/home/olili/.local/lib/python3.6/site-packages/matplotlib/backends/_backend_pdf_ps.py:62: RuntimeWarning: Glyph 12459 missing from current font.
  font.set_text(s, 0.0, flags=ft2font.LOAD_NO_HINTING)


Length of Cases Table[(140,)]
Length of Stocks Table[(196,)]
Length of Weather Table[(140,)]
Length of Tracks Table[(100,)]
Length of Artists Table[(100,)]


<Figure size 432x288 with 0 Axes>

<Figure size 2520x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 1800x1800 with 0 Axes>